In [1]:
import requests
import time
from bs4 import BeautifulSoup 
from selenium import webdriver
import re
from db_context import insert_one
headers = {"User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36 OPR/99.0.0.0'}
#&index [1-7]
scratch_list_url = 'https://www.lotto.pl/api/lotteries/zdrapki?name=&status=&yearSalesEnd=&page=70&scratchStatuses=Standard,Novelity,LastChance,Announcement,SalesEnd,Debut&index=1&size=12&sort=DateSalesUtc&order=DESC'

None


In [3]:
#read info about newest scratch cards
response = requests.get(scratch_list_url, headers=headers)
print(response.status_code, response.reason)
scratch_cards_data = response.json()

200 OK


In [14]:
def get_scratch_card_info(scratch_card_name, scratch_card_url):
    

    #read info about single scratch_card
    driver = webdriver.Chrome()
    driver.get('https://www.lotto.pl'+scratch_card_url)
    content = driver.page_source
    driver.close()

    soup = BeautifulSoup(content)
    strongs = soup.find_all('strong')[9].get_text().replace('\xa0', '')
    pot, circulation = re.findall('(\d+)',strongs)

    scratch_card = {'name': scratch_card_name,
                    'url': scratch_card_url,
                    'pot': pot,
                    'circulation': circulation,
                    'win_values': [],
                    'n_wins': [],
                    'wins_left': []
                    }

    table = soup.select('table td')

    scratch_card['win_value'] = [ table[i].get_text().replace('\xa0', '').replace('zł', '').strip() for i in range(3, len(table),3)]
    scratch_card['n_wins'] = [table[i].get_text() for i in range(4, len(table),3) ]
    scratch_card['wins_left'] = [table[i].get_text() for i in range(5, len(table),3) ]
    return scratch_card



In [19]:
print(scratch_cards_data['items'][0]['result']['name'])

MIĘTOWA KASA


In [15]:
name = scratch_cards_data['items'][0]['result']['name']
url = scratch_cards_data['items'][0]['meta']['url']

result = get_scratch_card_info(name, url)

print(result)




{'name': ('MIĘTOWA KASA',), 'url': '/zdrapki/katalog-zdrapek/mietowa-kasa,1323', 'pot': '22438500', 'circulation': '21000000', 'win_values': [], 'n_wins': ['14', '210', '5600', '42000', '315000', '210000', '577500', '3832500'], 'wins_left': ['14', '210', '5600', '42000', '315000', '210000', '577500', '3832500'], 'win_value': ['44000', '1000', '100', '50', '20', '10', '5', '2']}


In [16]:
print(insert_one(result))

64b9694fe93ffd8249b61b10
